IMPORTAR LIBRERIAS

In [41]:
import pandas as pd 
import json
from pandas import json_normalize
import ast
import re
import numpy as np



CARGA DE DATOS

In [42]:
df = pd.read_csv("./Movies/movies_dataset.csv")
df_2 = pd.read_csv("./Movies/credits.csv")

C:\Users\argen\AppData\Local\Temp\ipykernel_13476\1457517589.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Movies/movies_dataset.csv")


FUNCIONES DE APOYO

In [43]:

def reparacion_cadenas(cadena ):
    if isinstance(cadena, str):
        try:
            return ast.literal_eval(cadena)  # Intentar evaluar la cadena sin reemplazo de comillas
        except (ValueError, SyntaxError):  # Si falla, limpiar la cadena
            # Reemplazar comillas simples por dobles para estandarizar el formato
            cadena_limpia= cadena.replace("'", "\"")
            try:
                return ast.literal_eval(cadena_limpia)  # Intentar nuevamente después de limpiar
            except (ValueError, SyntaxError):
                return None  # Si todavía falla, devolver None para revisión
    return cadena  # Devolver los valores no string (como NaN) tal cual

DESANIDADO DE COLUMNAS DEL DATASET. Al revisar los datos, decidi trabajar con columnas de listas. Columnas como "belong_to colection que no me seran utiles, no las trabajare

In [44]:
df.shape

(45466, 24)

In [45]:
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [46]:
df.drop(columns='belongs_to_collection', inplace= True) #ELIMINAR COLUMNA "belongs_to_collection"

In [47]:
df["genres"].isnull().sum()

np.int64(0)

In [48]:
# Desanidado o arreglo de columna "genre", convirtiendola en una columna de listas de genero
df["genres"] = df["genres"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df['genres'] = df['genres'].apply(ast.literal_eval)
df['genres'] = df['genres'].apply(lambda x: [item['name'] for item in x])

In [49]:
# Desanidado o arreglo de columna "production_companies", convirtiendola en una columna de listas de productoras
df["production_companies"] = df["production_companies"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df['production_companies'] = df['production_companies'].apply(reparacion_cadenas)
df['production_companies'] = df['production_companies'].apply(
    lambda x: [item['name'] for item in x] if isinstance(x, list) and all(isinstance(i, dict) for i in x) else x)

In [50]:
# Desanidado o arreglo de columna "production_countries", convirtiendola en una columna de listas de paises_de_produccion
df["production_countries"] = df["production_countries"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df['production_countries'] = df['production_countries'].apply(reparacion_cadenas)
df['production_countries'] = df['production_countries'].apply(
    lambda x: [item['name'] for item in x] if isinstance(x, list) and all(isinstance(i, dict) for i in x) else x)

In [51]:
# Desanidado o arreglo de columna "spoken_languages", convirtiendola en una columna de listas de idiomas_hablados
df["spoken_languages"] = df["spoken_languages"].apply(lambda x: x.replace("'", '"') if pd.notna(x) else x)
df['spoken_languages'] = df['spoken_languages'].apply(reparacion_cadenas)
df['spoken_languages'] = df['spoken_languages'].apply(
    lambda x: [item['name'] for item in x] if isinstance(x, list) and all(isinstance(i, dict) for i in x) else x)


TRANSFORMACIONES

Valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [52]:
#Verificar si tiene nuloss y cuantos
print(df.revenue.isnull().sum())
print(df.budget.isnull().sum())

6
0


In [53]:
#rellenar los nulos con cero de la columna 'revenue'
df.loc[df.revenue.isnull()] = 0

Los valores nulos del campo release date deben eliminarse.

In [54]:
#verificar que la colunma 'release_date' tenga nulos y ceros
print(df['release_date'].isnull().sum())
print(df['release_date'].value_counts()[0])

84
6


In [55]:
s = df[df['release_date'] == 0]

In [56]:
s.head()

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19729,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0,0,0,0.0,0.0
19730,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0,0,0,0.0,0.0
29502,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0,0,0,0.0,0.0
29503,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0,0,0,0.0,0.0
35586,0,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0,0,0,0,0,0.0,0.0


In [57]:
#Eliminar filas sin datos
df = df[(df != 0).any(axis=1)]
#Eliminar filas valores nulos de columna 'release_date'
df.dropna(subset=['release_date'], inplace=True)

In [58]:
print(df['release_date'].isnull().sum()) #Re-Verificacion de nulos 

0


Camnio de Formato en las columnas con fecha ('release_date')

In [59]:
#Cambiar el tipo de dato de la columna 'release_date'
df['release_date'] =pd.to_datetime(df['release_date'])
#Crear la columna 'release_year'
df['release_year']= df['release_date'].dt.year

Creacion de la columna con el retorno de inversión ('return'), con los campos revenue y budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [60]:
#Cambio del tipo de dato de la columna 'budget'
df['budget'] = df['budget'].astype(float)
#Creacion de la columna Retorno de Inversion ('return')
df['return'] = ((df['revenue'] / df['budget'].replace(0, np.nan)).round(2)).fillna(0)

Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [61]:
df.drop(columns= ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

In [62]:
#Cambio de tupo de datos de otras columnas
df = df.astype({'id': int, 'popularity': float})

DESANIDADO DE COLUMNAS DEL DATASETS CREDITS

In [63]:
# Desanidado de Columa cast 

df_2["cast"] = df_2["cast"].apply(reparacion_cadenas)
df_2["cast"] = df_2["cast"].apply(lambda x: [item['name'] for item in x] if isinstance(x, list) and all(isinstance(i, dict) for i in x) else x)
df_2['cast'] = df_2['cast'].apply(lambda x:set(x))
df_2['cast'] = df_2['cast'].apply(lambda x:list(x))

In [64]:
df_2["crew"] = df_2["crew"].apply(reparacion_cadenas)
df_2["crew"] = df_2["crew"].apply(
    lambda x: [item['name'] for item in x if item['job']=='Director'] if isinstance(x, list) and all(isinstance(i, dict) for i in x)  else x)

AGRUPACION DE DATASETS

In [65]:
df = pd.merge(df, df_2, on="id")

In [66]:
df.shape

(45451, 21)

RENOMBRAR COLUMNAS

In [67]:
df.rename(columns={'crew':'director', 'cast': 'actors',}, inplace=True)

MODIFICAR COLUMNAS DE LENGUAJE

In [68]:
#SOLO VAMOS A TRABAJAR CON PELICULAS EN IDIOMA ORIGINAL EN INGLES
df.loc[df['original_language'].isnull() & df['spoken_languages'].apply(lambda x: 'English' in x),'original_language']='en'
df.dropna(subset=['original_language'], inplace=True)
df = df[df['original_language'] == 'en'].reset_index(drop=True)

In [69]:
#Ver que valos nulos quedan en al Datasets
df.isnull().sum()

budget                      0
genres                      0
id                          0
original_language           0
overview                   68
popularity                  0
production_companies      241
production_countries        1
release_date                0
revenue                     0
runtime                     5
spoken_languages            0
status                     67
tagline                 14321
title                       0
vote_average                0
vote_count                  0
release_year                0
return                      0
actors                      0
director                    0
dtype: int64

In [70]:
#Valores Nulos en la Columna "overview"
over_missing = df.loc[df["overview"].isnull(), "overview"]

In [71]:
over_missing

702      NaN
907      NaN
1971     NaN
3715     NaN
8377     NaN
        ... 
31592    NaN
31718    NaN
31744    NaN
32033    NaN
32178    NaN
Name: overview, Length: 68, dtype: object

In [72]:
#Rellenar los Valor nulos con cadenas vacias
df.loc[df["overview"].isnull(),"overview"] = ""
df.loc[df["production_companies"].isnull(), "production_companies"]= ""

In [73]:
#Valores nulos de la columna "status"
status_missing = df.loc[df["status"].isnull(), ["status",'release_date']]

In [74]:
status_missing

,status,release_date
170,NaN,1995-08-25
682,NaN,1995-01-01
1218,NaN,1997-01-01
3715,NaN,2001-06-06
5404,NaN,2002-09-09
...,...,...
31659,NaN,1994-08-15
32038,NaN,2009-09-25
32055,NaN,2005-09-30
32089,NaN,1990-11-30


In [75]:
#Rellenar los datos vacios de estatua a 'Released' si existe fecha de lanzamiento
df.loc[df['release_date'].notnull(), 'status'] = 'Released'

In [76]:
df.isnull().sum()

budget                      0
genres                      0
id                          0
original_language           0
overview                    0
popularity                  0
production_companies        0
production_countries        1
release_date                0
revenue                     0
runtime                     5
spoken_languages            0
status                      0
tagline                 14321
title                       0
vote_average                0
vote_count                  0
release_year                0
return                      0
actors                      0
director                    0
dtype: int64

In [77]:
#ELIMINACION DE COLUMNAS INNECESARIAS
df.drop(columns=["tagline","spoken_languages", "production_countries", "runtime"],  inplace=True)


GENERAR DATASETS PARA CONSULTAS

In [78]:
df.shape

(32256, 17)

In [79]:
df.to_csv('./Movies/dataset.csv', index=False)